<a href="https://colab.research.google.com/github/alfiannnas/tbc-detection-app/blob/main/TBC_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TBC Detection using CNN

## Import Library

In [3]:
import shutil
import kagglehub

## Import Dataset

In [5]:
# Download latest version
path = kagglehub.dataset_download("tawsifurrahman/tuberculosis-tb-chest-xray-dataset")

print("Path to dataset files:", path)

100%|██████████| 663M/663M [00:10<00:00, 67.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset/versions/3


In [6]:
# Path Dataset from KaggleHub
src_path = "/root/.cache/kagglehub/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset/versions/3"

# Destination Path
dst_path = "/content/dataset"

# Copy Dataset from KaggleHub into Destination Path
shutil.copytree(src_path, dst_path, dirs_exist_ok=True)

print("Dataset copied to:", dst_path)


Dataset copied to: /content/dataset
